In [31]:
import pandas as pd
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets
import pickle
from nltk.tokenize import word_tokenize
from sklearn.decomposition import NMF, TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve, auc, fbeta_score
from sklearn.model_selection import learning_curve, KFold, GridSearchCV, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.preprocessing import normalize
import re
from unidecode import unidecode

import matplotlib.style as style
%matplotlib inline
plt.style.use('ggplot')

In [32]:
with open('data/cleaned_df.pkl', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [33]:
df.head()

,song_owner,song_title,song_element,verse,artist
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino


In [34]:
model_df = df.copy()

model_df = model_df[model_df['artist'] == 'Kendrick Lamar']

In [35]:
model_df['verse_words'] = model_df['verse'].copy()
model_df['verse_words'] = model_df['verse_words'].str.lower()
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.replace(',',''))
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.split(' '))

In [36]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words
1283,Drake,Buried Alive (Interlude),[Verse 1: Kendrick Lamar],"Lookin' in the mirror, I'm embarrassed, I'm fe...",Kendrick Lamar,"[lookin', in, the, mirror, i'm, embarrassed, i..."
1806,Drake,Buried Alive (Interlude),[Verse 1: Kendrick Lamar],"Lookin' in the mirror, I'm embarrassed, I'm fe...",Kendrick Lamar,"[lookin', in, the, mirror, i'm, embarrassed, i..."
2890,Kendrick Lamar,Look Out For Detox,[Intro],"Uh, shoutout to ScHoolboy Q, He uglier than a ...",Kendrick Lamar,"[uh, shoutout, to, schoolboy, q, he, uglier, t..."
2891,Kendrick Lamar,Look Out For Detox,[Verse],"Tire marks, tire marks, Finish line with the t...",Kendrick Lamar,"[tire, marks, tire, marks, finish, line, with,..."
2892,Kendrick Lamar,HUMBLE.,[Intro],"Nobody pray for me, It's been that day for me,...",Kendrick Lamar,"[nobody, pray, for, me, it's, been, that, day,..."


In [37]:
from nltk.tokenize import word_tokenize
model_df['tokenize_verse'] = model_df['verse'].copy()
model_df['tokenize_verse'] = model_df['tokenize_verse'].str.lower()
model_df['tokenize_verse'] = model_df['tokenize_verse'].apply(lambda x: word_tokenize(x))

In [38]:
model_df['verse'] = model_df['verse'].str.lower()
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace("'",''))
model_df['verse'] = model_df['verse'].apply(lambda x: "".join([c if c.isalnum() else " " for c in x ]))
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace('  ',' '))

In [39]:
stopwords = ["yeah", "uh", "huh", "aw", "oh", "la", "ayy",
            "na", "ha", "ay", "skrrt",
            "em", "bla", "woo", "wu", "ya", "wala", "woop", "wrarrr", "ooh",
            "ba", "duh", "du", "1", "2", "3", "4" ,"5" ,"6", "7", "8", "9", "0"]

for index, row in model_df.iterrows():
    querywords = row['verse'].split()

    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    row['verse'] = ' '.join(resultwords)

model_df.verse[:5]

1283    lookin in the mirror im embarrassed im feelin ...
1806    lookin in the mirror im embarrassed im feelin ...
2890    shoutout to schoolboy q he uglier than a mothe...
2891    tire marks tire marks finish line with the tir...
2892    nobody pray for me its been that day for me wa...
Name: verse, dtype: object

In [40]:
verses = list(model_df.verse.values)

In [41]:
verses[:10]

['lookin in the mirror im embarrassed im feelin like a suicidal terrorist react like an infant whenever you are mentioned mind over matter never worked for my nemesis im in the matter of man arm wrestlin hands i was dealt when i said the music business was all i needed when i got it i was greeted by an alien that said last year that she slept with a canadian that gave him an addiction thatd keep him in mercedes benz bright lights and rihanna as a lady friend my vice is similar women love when youre my type and youre winnin from everything that your palm write put her in the palms hotel sin city devil in a dress platinum chanel live the ambiance all cause the audience one day said i would do it so instead of a verse bein read ima go and get some head off the strength of my music i tell a bad bitch yo ass too fat capitalize that and your weave look good with the indian tracks trackin device on your used series i dont call back just blame it on your canadian the same day we say were in th

In [42]:
all_verses_all_rappers = ','.join(verses)

all_verses_all_rappers[:100]

'lookin in the mirror im embarrassed im feelin like a suicidal terrorist react like an infant wheneve'

In [43]:
file = all_verses_all_rappers

In [44]:
# Read in text and change unicode characters to ASCII
import unidecode
import string
import random
import re

all_characters = string.printable
n_characters = len(all_characters)

file_len = len(file)
print(f'file_len = {file_len}')

file_len = 712344


In [45]:
chunk_len = 400

def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

 a hindrance surrenduring ninja turtles pizza and sega genesis heard it over my bunk bed ear hustling something my papa said that caught my attention his eyes red hes smoking on marijuana mixing the two eleven he said im seven going on sixty seven the day i turned seventeen is the day that a lesson inside my lifetime come across a clothes line every stick is a co sign everything in the streets got 


In [46]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        output = self.encoder(input.view(1, -1))
        output, hidden = self.gru(output.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        
        return output, hidden
    
    def init_hidden(self):
        return Variable(torch.randn(self.n_layers, 1, self.hidden_size))

In [47]:
def char2tensor(string):
    tensor = torch.zeros(len(string)).long()
    for char in range(len(string)):
        if string[char] in all_characters:
            tensor[char] = all_characters.index(string[char])
        else:
            tensor[char] = 94 #predict space if character unknown
        
    return Variable(tensor)

# Let's see it in action.
print(char2tensor('Metis0123abczABC'))

tensor([48, 14, 29, 18, 28,  0,  1,  2,  3, 10, 11, 12, 35, 36, 37, 38])


In [48]:
def random_training_set():
    chunk = random_chunk()
    inp = char2tensor(chunk[:-1])
    target = char2tensor(chunk[1:])
    return inp, target

In [49]:
def evaluate(model, prime_str='A', predict_len=100, temperature=0.8):
    
    hidden = model.init_hidden()
    prime_input = char2tensor(prime_str)
    predicted = prime_str
    
    # use priming string to build up hidden state
    
    for p in range(len(prime_str) - 1):
        _, hidden = model(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = model(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char2tensor(predicted_char)
        
    return predicted

In [50]:
# helper function
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s/60)
    s -= m*60
    return '%dm %ds' % (m, s)

In [51]:
# The actual training part
def train(inp, target):
    hidden = model.init_hidden()
    model.zero_grad()
    loss = 0

    for char in range(chunk_len):
        output, hidden = model(inp[char], hidden)
        loss += criterion(output, target[char].unsqueeze(0))

    loss.backward()
    model_optimizer.step()

    return loss.data.item() / chunk_len

In [52]:
# parameters
# n_epochs = 1000
# print_every = 100
# plot_every = 10
# hidden_size = 256
# n_layers = 2
# learning_rate = 0.001

# # model declaration
# model = RNN(n_characters, hidden_size, n_characters, n_layers)
# model_optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# criterion = nn.CrossEntropyLoss()

# start = time.time()
# all_losses = []
# loss_avg = 0

# for epoch in range(1, n_epochs + 1):
#     loss = train(*random_training_set())       
#     loss_avg += loss

#     if epoch % print_every == 0:
#         print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
#         print(evaluate(model, 'A ', 100), '\n')

#     if epoch % plot_every == 0:
#         all_losses.append(loss_avg / plot_every)
#         loss_avg = 0

In [53]:
# with open('models/kendrick_rnn_generator.pkl', 'wb') as picklefile:
#     pickle.dump(model, picklefile)

In [54]:
# import matplotlib.pyplot as plt
# import matplotlib.ticker as ticker
# %matplotlib inline

# plt.figure()
# plt.plot(all_losses)

In [55]:
# Evaluate model from this notebook
print(evaluate(model, 'To be or not to be: ', predict_len=200, temperature=0.5))

To be or not to be: of a boo he motherfuck dont lovin i know and the polication it ass and the song bless all all i compless and you know when the cong my song compton when i dont key bout hall and i problessin of my bea


In [56]:
# Evaluate model from this notebook
print(evaluate(model, 'I miss you girl come back ', predict_len=200, temperature=0.5))

I miss you girl come back i bank i got then high dont king you got a back accever that with made i know the cont and on my see what a bitch on heart some when i dont dont dont back me when you know with down when you at to the


In [57]:
# Evaluate model from this notebook
print(evaluate(model, "Started from the bottom", predict_len=200, temperature=0.4))

Started from the bottom thats i was pay and compton the copper i how a can dont dont band i can my pint king me all the song me a boy for the say the can be and the she some day in the song i know what i know when i dont so


In [58]:
# Evaluate model from this notebook
print(evaluate(model, "I'm the realest rapper alive", predict_len=200, temperature=0.4))

I'm the realest rapper alive me be of my can be and the compless boy i got the dont dont can head i see and the and dont dont dont know it and you dont dont i song man dont king i can in the complicated i real i can down poppere


In [59]:
# Evaluate model from this notebook
print(evaluate(model, "Smoke a lot of weed", predict_len=200, temperature=0.4))

Smoke a lot of weed he dont sing i was the song a band you sing you know when you song me the dont got the bard a song it back and the can a king my song you and be song bout i can a cond the song me i cong me i dont be


In [60]:
# Evaluate model from this notebook
print(evaluate(model, "Baby come", predict_len=200, temperature=0.4))

Baby come i can be the king the see when i know when i can a can good a bear the back all what you see i was all and back and that i know you sing a know what you a black i was to be sing the seen the king i d


In [61]:
with open('models/kendrick_rnn_generator.pkl', 'rb') as picklefile:
    kendrick_rnn = pickle.load(picklefile)

In [66]:
prompts = ['I miss you girl come back ', 'To be or not to be: ', "Started from the bottom", "I'm the realest rapper alive",
           "Smoke a lot of weed", "Baby come", 'Date a scientist', 'Butter my biscuit', 'Scramble my ', 'I sleep ',
           'Green eggs and ', 'Rollin in my', 'Drop top', 'Math is fun', 'Sentences', 'Coffee in the', 'Table that',
           'Homies', 'From the streets', 'Maserati', 'Air force one', 'President', 'Trump']

kendrick_rnn_verses = []

for i in range(len(prompts)):
    kendrick_rnn_verses.append(evaluate(model, prompts[i], predict_len=200, temperature=0.5))
    
    
kendrick_rnn_verses

['I miss you girl come back to make you know you was know she all i can hear complicating the for my put the point i was the say who and the bang you dont complin a find the pornice the hood dont i see for that where you a see a',
 'To be or not to be: and my song of my from the so you sing the cause the care on his dont know you want my song i sing you said a be song all broke when my walk you power the could dont dont mine when i can and you and h',
 'Started from the bottom a tist be for king and that so care the people with the song to can doo and be i see and i can and back to my know i am when you sing and be i cong be actin a complice the martin who on my child want',
 "I'm the realest rapper alive i sing my dont the antion i can bitch for me that my dont all me a king and my care to can and i can day you know i know when the world the street to problead i dont good a call me that want was bout",
 'Smoke a lot of weed have back a had the can bullest and you king a bout the for t

In [70]:
columns = ['generated_verses']

kendrick_generated_df = pd.DataFrame(kendrick_rnn_verses, columns=columns)

In [72]:
kendrick_generated_df['artist'] = 'Kendrick Lamar'

kendrick_generated_df

,generated_verses,artist
0,I miss you girl come back to make you know you...,Kendrick Lamar
1,To be or not to be: and my song of my from the...,Kendrick Lamar
2,Started from the bottom a tist be for king and...,Kendrick Lamar
3,I'm the realest rapper alive i sing my dont th...,Kendrick Lamar
4,Smoke a lot of weed have back a had the can bu...,Kendrick Lamar
5,Baby come on the can had bout the parce the se...,Kendrick Lamar
6,Date a scientist and the song not and the borg...,Kendrick Lamar
7,Butter my biscuition the and how a sin coorder...,Kendrick Lamar
8,Scramble my sing so the flow me so who a compr...,Kendrick Lamar
9,I sleep a wait of king i m the she nigga it mo...,Kendrick Lamar


In [73]:
with open('data/kendrick_generated_raps.pkl', 'wb') as picklefile:
    pickle.dump(kendrick_generated_df, picklefile)